In [1]:
from types import MethodType
from escalerasyserpientes import EscalerasSerpientes
import copy
import numpy as np

In [21]:
es=EscalerasSerpientes([80,100],[23,37,45,67,89],-0.001)

In [28]:
print(es.states)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]


In [22]:
#q value algorithm
win_per = 0
prom_step = []
alpha = 0.5
gamma = 1

# Arreglos para los valores
q_value = dict.fromkeys(es.q_values , 0.0)
#Para 500 episodio
for i in range(1000):
    num_steps = 0
    state = es.states[0]
    act_arg = np.array([es.q_values[(state, act)] for act in es.allowed_actions[state]])
    action = es.allowed_actions[state][np.argmax(act_arg)]
    
    continue_episode = True
    while continue_episode:

        if state in es.snakes:
            state = es.state_values[es.snakes[state]]
        elif state in es.stairs:
            state = es.state_values[es.stairs[state]]

        # Tomar acción dependiendo de epsilon y de la acción con mayor recompensa
        epsilon = 0.1
        if np.random.uniform()<epsilon:
            A = np.random.randint(0,2)
            actions = es.allowed_actions[1]
            action = actions[A]
            new_state, reward, _, done = es.step(state, action, random=True)

        else:
            new_state, reward, _, done = es.step(state, action, random=True)

        if type(new_state) is str:
            q_value_next_step = 0
        else:
            action_arg = np.array([es.q_values[(new_state, act)] for act in es.allowed_actions[state]])
            #print(action_arg)
            new_action = es.allowed_actions[new_state][np.argmax(action_arg)]
            q_value_next_step = es.q_values[(new_state,new_action)]






        es.q_values[(state, action)] += alpha*(reward + gamma*q_value_next_step - es.q_values[(state,action)])

        state = new_state
        action = new_action
        #print(action, state)
        num_steps+=1        

        if done:
            continue_episode = False
            if state == 'Azul':
                win_per+=1
    prom_step.append(num_steps)
    #print('-'*30)

In [6]:
es.q_values

{(1, 'Ad'): 0.9999999917698957,
 (1, 'At'): 0.9999999917710964,
 (2, 'Ad'): 0.9995365267412872,
 (2, 'At'): 0.9999999917710964,
 (3, 'Ad'): 0.999768033106504,
 (3, 'At'): 0.9999999917710964,
 (4, 'Ad'): 0.999998803597419,
 (4, 'At'): 0.9999999917710964,
 (5, 'Ad'): 0.999534606444224,
 (5, 'At'): 0.9999999917710964,
 (6, 'Ad'): 0.9999094412027953,
 (6, 'At'): 0.9999999917710964,
 (7, 'Ad'): 0.9999980197283078,
 (7, 'At'): 0.9999999917710964,
 (8, 'Ad'): 0.0,
 (8, 'At'): 0.0,
 (9, 'Ad'): 0.9999999999999998,
 (9, 'At'): 0.9998768069051513,
 (10, 'Ad'): 0.9999999999999998,
 (10, 'At'): 0.9996813627424784,
 (11, 'Ad'): 0.9999999999999998,
 (11, 'At'): 0.9999971685908604,
 (12, 'Ad'): 0.9999999999999998,
 (12, 'At'): 0.9996523930119834,
 (13, 'Ad'): 0.9999999999999998,
 (13, 'At'): 0.9999782689374064,
 (14, 'Ad'): 0.9999999999999998,
 (14, 'At'): 0.9995215939140076,
 (15, 'Ad'): 0.9999999999999999,
 (15, 'At'): 0.9999999999999998,
 (16, 'Ad'): 0.9999999999999999,
 (16, 'At'): 0.9999999999999

In [200]:
state = 99
print(type (es.allowed_actions[state]))
a = ([q_value[(state,a)] for a in es.allowed_actions[state]])
print(np.argmax(a),state)
print(es.allowed_actions[state][np.argmax(a)])
#print(es.q_values(state,a) for a in es.allowed_actions[state])


<class 'list'>
0 99
Ad


In [24]:
print(win_per*100/1000)  

67.7


In [25]:
print(prom_step)

[21, 18, 17, 17, 35, 17, 43, 13, 20, 21, 96, 35, 28, 30, 20, 18, 53, 19, 8, 13, 44, 47, 37, 69, 82, 15, 20, 15, 60, 18, 22, 4, 17, 7, 29, 61, 79, 43, 25, 53, 17, 7, 35, 12, 38, 17, 20, 18, 21, 15, 22, 38, 49, 19, 86, 14, 26, 19, 44, 24, 97, 13, 134, 13, 96, 77, 15, 39, 98, 101, 40, 19, 31, 32, 72, 50, 41, 136, 159, 22, 65, 31, 54, 38, 35, 89, 308, 169, 62, 247, 60, 172, 70, 18, 27, 33, 31, 181, 73, 16, 72, 15, 78, 25, 54, 29, 19, 24, 205, 50, 20, 84, 71, 53, 41, 172, 37, 127, 36, 24, 61, 85, 40, 70, 48, 45, 46, 199, 41, 78, 156, 56, 103, 15, 125, 43, 12, 75, 47, 25, 64, 8, 15, 142, 48, 26, 59, 19, 32, 25, 20, 120, 232, 49, 34, 173, 101, 60, 39, 25, 30, 17, 13, 60, 28, 258, 73, 23, 135, 30, 48, 125, 19, 19, 52, 26, 82, 51, 35, 137, 248, 110, 34, 69, 10, 34, 101, 25, 143, 35, 104, 71, 94, 28, 65, 76, 12, 90, 55, 115, 335, 217, 57, 33, 34, 19, 101, 11, 88, 73, 48, 119, 91, 64, 98, 122, 58, 178, 161, 140, 23, 99, 131, 25, 310, 64, 187, 65, 46, 37, 45, 44, 250, 101, 77, 44, 86, 67, 35, 59, 

In [23]:
sum(prom_step)/1000

66.185